# Problem 4

In this problem, we simply finetune a BERT model (not pretrained) on RTE dataset, and then finetune a BERT model (pretrained) on RTE dataset.

**IMPORTANT NOTES**:
- Please make sure that you have already read the part of hw5 pdf that corresponds to this problem. This is very important.
- At the end of the hw5, you will need to submit a zip folder containing three things. The instruction is also included in the first paragraph of the hw5 pdf.
  - (1) The writeup pdf containing your solutions to Problems 1, 2, 3, 4, 5. Yes, there're things you need to respond in your writeup (see hw5 pdf).
  - (2) The downloaded colab corresponding to Problem 4.
  - (3) The downloaded colab corresponding to Problem 5.

Some imports and data downloading

In [1]:
!pip install transformers

In [2]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
import torch.nn as nn 
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModel, BertPreTrainedModel, AutoConfig, BertModel
from transformers import GlueDataTrainingArguments 
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

In [3]:
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks RTE
!pip install transformers

fatal: destination path 'transformers' already exists and is not an empty directory.
	Completed!


In [4]:
!ls glue_data/RTE/

dev.tsv  test.tsv  train.tsv


In [24]:
model_name = "bert-base-uncased"

data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./glue_data/RTE")
training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    #evaluate_during_training=True,
    evaluation_strategy = 'steps',
    eval_steps = 50,
    output_dir="./models/rte",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=0.00001,
    num_train_epochs=10
)
set_seed(42)
num_labels = glue_tasks_num_labels[data_args.task_name]

### From non-pretrained BERT

TODO:
- Complete the following three lines such that ```tokenizer``` and ```config``` and ```bert_model``` corresponds to the ```model_name``` we defined in the above cells. 
- IMPORTANT: make sure that the BERT model does not load pretrained weights!
- Hint: https://huggingface.co/transformers/model_doc/auto.html and other relevant Hugging Face documentations. Consider using the tools we imported in the first cell. More hints: it's okay to use ```from_pretrained``` in the first two lines, depending on what class you use.

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
bert_model = BertModel(config)

In [26]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

TODO:
- Complete the forward function of the following class such that the model can do finetuning on RTE dataset.
- For more instructions, please refer to the hw5 pdf.

In [27]:
class SequenceClassificationBERT(nn.Module):
      
    def __init__(self, config, bert_model):
        super().__init__()
        self.config = config
        self.num_labels = config.num_labels
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.bert = bert_model

        self.pooling = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.Tanh(),
        )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # make sure that all the arguments in the forward() function is used
        # somewhere in the code

        ##### 

        # TODO
        outputs = self.bert(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask,
                               inputs_embeds=inputs_embeds,
                               output_attentions=output_attentions,
                               output_hidden_states=output_hidden_states,
                               return_dict=return_dict,
                               )
        last_hidden_state = outputs[0]
        h_cls = last_hidden_state[:, 0]
        representation = self.pooling(h_cls)
        representation = self.dropout(representation)
        logits = self.classifier(h_cls)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        #####
        # do not change the lines below, so make sure your code works for the
        # lines below
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output


In [28]:
model = SequenceClassificationBERT(config=config, bert_model=bert_model)

TODO:
- Print out the number of trainable parameters in the BERT model. This can be done in one line. Please feel free to look up resources online. We also briefly touched upon relevant materials in Lab 3, but here, make sure you only count the number of trainable parameters.

In [29]:
from transformers.modeling_utils import ModuleUtilsMixin

In [30]:
 n_parameters = ModuleUtilsMixin.num_parameters(model, only_trainable=True)# TODO
 print(n_parameters)

110074370


In [31]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


Now we train the model. Please make sure to read the pdf instructions. When you report results in the pdf writeup, make sure you report the mean and std of >=3 runs with different random seeds. Consider using ```set_seed(some number)``` before the below cell, before each run.

Make sure in each run, you're picking the best validation accuracy. We're using Trainer instead of the normal training loop which we have seen many many times earlier in the semester. In the trainer, we need to specify ```num_train_epochs``` (in ```training_args```) which we defined above. Please feel free to modify ```training_args``` such that:
- The learning rate is small (around 0.00001).
- Your model doesn't have large improments on validation accuracy anymore, at the end of training. The expected behavior is that the final validation accuracy won't be much better than chance.

We provided part of an example log below, but you may be able to get better accuracy. Again, make sure this run corresponds to using an non-pretrained BERT.

In [32]:
training_args.num_train_epochs = 10

In [15]:
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

In [ ]:
set_seed(42)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Acc
50,0.768459,0.695390,0.527076
100,0.751764,0.730771,0.527076
150,0.712441,0.692730,0.523466
200,0.702328,0.691924,0.519856
250,0.703555,0.691637,0.523466
300,0.694853,0.699265,0.490975
350,0.680095,0.730020,0.465704
400,0.674226,0.724666,0.490975
450,0.665272,0.731429,0.534296
500,0.640493,0.805318,0.472924


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 10.0,
 'eval_acc': 0.5234657039711191,
 'eval_loss': 0.8835858702659607}

In [ ]:
set_seed(1997)
model = SequenceClassificationBERT(config=config, bert_model=bert_model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Acc
50,0.628642,1.058475,0.501805
100,0.595582,1.080957,0.541516
150,0.498669,1.094595,0.509025
200,0.458384,1.092436,0.516245
250,0.506181,1.130333,0.501805
300,0.387762,1.336087,0.487365
350,0.384198,1.310127,0.501805
400,0.331406,1.424132,0.501805
450,0.278065,1.619184,0.494585
500,0.354178,1.509198,0.498195


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 10.0,
 'eval_acc': 0.5054151624548736,
 'eval_loss': 1.7034538984298706}

In [ ]:
set_seed(1126)
model = SequenceClassificationBERT(config=config, bert_model=bert_model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Acc
50,0.720975,0.843542,0.494585
100,0.332882,1.657962,0.505415
150,0.300393,1.604890,0.490975
200,0.213642,2.024510,0.501805
250,0.228073,1.988131,0.509025
300,0.233413,2.189665,0.501805
350,0.236262,2.123801,0.476534
400,0.179557,2.060687,0.505415
450,0.198900,2.310726,0.476534
500,0.201956,2.383541,0.494585


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 10.0,
 'eval_acc': 0.5054151624548736,
 'eval_loss': 2.5044546127319336}

In [ ]:
best_val_acc = [0.534296, 0.541516, 0.509025]
print('mean across 3 runs:', np.mean(best_val_acc))
print('std across 3 runs:', np.std(best_val_acc))

mean across 3 runs: 0.5282789999999999
std across 3 runs: 0.01393005113654172


### From pretrained BERT

Now, let's do the above experiments using a pretrained BERT!

TODO:
- Complete the following three lines such that ```tokenizer``` and ```config``` and ```bert_model``` corresponds to the ```model_name``` we defined in the above cells. 
- IMPORTANT (different from the TODO a few cells above): make sure that the BERT model below loads the pretrained weights!

In [34]:
model_name

'bert-base-uncased'

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

In [35]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [36]:
model = SequenceClassificationBERT(config=config, bert_model=bert_model)

TODO:
- Similarly, we train the model. For more instructions, please see the TODO cells above (i.e., the TODO corresponding to training the model, when we're not loading weights into BERT), as well as the hw5 pdf.

In [37]:
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

In [38]:
set_seed(42)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Acc
50,0.700295,0.674470,0.599278
100,0.672086,0.677756,0.581227
150,0.627965,0.635070,0.642599
200,0.541012,0.658559,0.638989
250,0.526751,0.660446,0.642599
300,0.441103,0.681910,0.642599
350,0.393155,0.751332,0.646209
400,0.362894,0.814175,0.624549
450,0.301450,0.827498,0.613718
500,0.261257,0.845414,0.635379


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 10.0, 'eval_acc': 0.631768953068592, 'eval_loss': 0.9995214343070984}

In [41]:
set_seed(1997)
bert_model = BertModel.from_pretrained(model_name)
model = SequenceClassificationBERT(config=config, bert_model=bert_model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Acc
50,0.691088,0.660004,0.617329
100,0.660243,0.689239,0.566787
150,0.605775,0.627486,0.660650
200,0.511932,0.666382,0.657040
250,0.497911,0.655082,0.631769
300,0.419525,0.673832,0.664260
350,0.363789,0.746073,0.657040
400,0.341183,0.806862,0.628159
450,0.279752,0.850157,0.642599
500,0.235952,0.837323,0.675090


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 10.0,
 'eval_acc': 0.6462093862815884,
 'eval_loss': 1.0077791213989258}

In [42]:
set_seed(1126)
bert_model = BertModel.from_pretrained(model_name)
model = SequenceClassificationBERT(config=config, bert_model=bert_model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Acc
50,0.688530,0.674750,0.548736
100,0.657139,0.673891,0.588448
150,0.608168,0.650459,0.610108
200,0.506686,0.667125,0.624549
250,0.477550,0.683766,0.638989
300,0.387487,0.710679,0.646209
350,0.318424,0.764403,0.638989
400,0.285782,0.799312,0.638989
450,0.215782,0.910586,0.638989
500,0.170453,0.904604,0.628159


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 10.0,
 'eval_acc': 0.6498194945848376,
 'eval_loss': 1.0695747137069702}

In [43]:
best_val_acc = [0.646209, 0.675090, 0.653430]
print('mean across 3 runs:', np.mean(best_val_acc))
print('std across 3 runs:', np.std(best_val_acc))

mean across 3 runs: 0.658243
std across 3 runs: 0.012271967160972992
